In [ ]:
#imports
import numpy as np
from Bio import Phylo
import pandas as pd

# Background

The purpose of this project is to investigate if codon bias is an indicator for phylogenetic relationships. If codon bias does prove to be an indicator, this could be used to increase understanding of the relationship between different organisms. A csv file containing the codon bias for many different organisms was taken from Kaggle (source?). TO make the project more manageable, rodents were selected to be the focus on the project. The data was analyzed by creating a data frame and using pandas techniques from BIOL300 such as reading in a file and indexing a data frame. To create a phylogenetic tree, the algorithm UPGMA from BIOL301 was used. 

# Part 1 - Preparing the Data

First, the csv file must be read into the file as a data frame. This was done using Pandas a skills from BIOL300. 

In [ ]:
#read file into a data frame
f = open('codon_usage.csv', 'r')
df = pd.read_csv(f);

The csv files is quite large, so it was decreased in size to be more manageable. Rodents were selected as the subject of this study, so all animals labeled as in the rodent kingdom were added to a new data frame. Then, since this data included codon bias for mitochondrial DNA, that was also removed to limit the scope of this project. 

In [ ]:
#parse out only columns labeled as 'rod' (rodents)
rodents_with_mit = df[df['Kingdom'] == 'rod']

#remove the mitocondrial DNA by using the DNAtype column
rodents = rodents_with_mit[rodents_with_mit['DNAtype'] == 0]

In [ ]:
#limit the number of rodents and print out dataframe
rodents = rodents[8:13]
rodents

The csv file contained codon bias across the entire genome. It was decided to normalize these percentages for each amino acid. This removed any amino acid bias between species and focuses just on the codon bias. To accomplish this, a dictionary was created to correlate each codon to a specific amino acid (source?). Multiple checks were performed to make sure all amino acids and codons were present and there were no typos. 

In [ ]:
#create dictionary of amino acids and corresponding codons
aas = {'Phe': ['UUU','UUC'], 'Leu': ['UUA','UUG','CUU','CUC','CUA','CUG'], 'Ile': ['AUU','AUC','AUA'], 'Met': ['AUG'], 'Val': ['GUU','GUC','GUA','GUG'],
       'Ser': ['UCU','UCC','UCA','UCG','AGU','AGC'], 'Pro': ['CCU','CCC','CCA','CCG'], 'Thr': ['ACU','ACC','ACA','ACG'], 'Ala': ['GCU','GCC','GCA','GCG'], 
       'Tyr': ['UAU','UAC'], '***': ['UAA','UAG','UGA'], 'Trp': ['UGG'], 'His': ['CAU','CAC'], 'Gln': ['CAA','CAG'], 'Asn': ['AAU','AAC'], 'Lys': ['AAA','AAG'],
       'Cys': ['UGU','UGC'], 'Arg': ['CGU','CGC','CGA','CGG','AGA','AGG'], 'Gly': ['GGU','GGC','GGA','GGG'], 'Asp': ['GAU','GAC'], 'Glu': ['GAA','GAG']}

In [ ]:
#check that all amino acids are in dictionary (20+stop = 21)
len(aas)

In [ ]:
#check that all codons are included in dictionary (should be 64)
count = 0
for i in aas:
    count += len(aas[i])
    
print(count)

Once the dictionary was completed and checked, it was used to standardize by amino acid. By iterating through the dictionary, it was established how often each codon was used to create a certain amino acid for each animal. These new percentages were then added to the data frame, replacing the former percentages which accounted for the whole genome. The data frame was then printed to make sure the data frame was updated

In [ ]:
#standarize by amino acid

#loop through each animal
for animal in rodents['SpeciesName']:
    species_bool = rodents['SpeciesName'] == animal
    species = rodents[species_bool]
    
    #loop through all amino acids
    for aa in aas:
        #initalize total percent of amino acid
        total = 0 
        
        #loop through codons and add up percentages
        for codon in aas[aa]:
            total += species[codon]
        
        #loop through codons, normalize by amino acid, and replace in data frame
        for codon in aas[aa]:
            rodents.loc[species_bool, [codon]] = species[codon]/total

In [ ]:
#print rodents to make sure it updated 
rodents

Once the data was cleaned and standardized by amino acid, a distance function was created to determine the distance between two species. It was decided to use a sum of squares. The squared difference between each codon was added together and the total difference returned to determine how different codon bias between species is.

In [ ]:
def species_diff(df, species1, species2):
    # finds the difference between the codon bias of two species
    
    # step a counting value to 0
    diff = 0
    
    animal1 = df[df['SpeciesName'] == species1]
    animal2 = df[df['SpeciesName'] == species2]
    
    #loop through all amino acids
    for aa in aas:
        #loop through all codons
        for codon in aas[aa]:
            #add the squared difference between the species codon bias squared
            diff += (animal1[codon].values[0] - animal2[codon].values[0])**2
    
    #return the total difference between two speices
    return diff         

# Part 2 - UPGMA

Now, the UPGMA algorithm must be implemented. We started with the algorithm developed in class. However, this version did not take into account edge weights when making the final tree, which we wanted to include. So, a simple Node class was created which could store all of this data. The left and right variables store tuples that point to the child node and the distance between the two nodes.

In [ ]:
class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.val = None

To test that the edge weights worked, the example from class was used. This uses sequences instead of codon bias, so we must first define a difference function for sequences. This function assumes that the sequences are the same length which is a safe assumption since this is just being used to test phylogenetic tree formation and not generalized to other sequences

In [ ]:
def differences(seq1, seq2):
    '''Counts the number of pairwise differences between
    two sequences'''
    
    #set count variable to zero
    count = 0
    
    #loop through the first sequence
    for i in range(len(seq1)):
        
        #if the amino acids don't match, increase the count variable by one
        if seq1[i] != seq2[i]:
            count += 1

    #once the entire sequence has been checked, return the count variable        
    return count

Next, all of the hemoglobin sequences must be defined. This information was taken from the notebook used in class

In [ ]:
human =   "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
chimp =   "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
gorilla = ".VLSPADKTNVKAAWGKVGAHAGDYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
cow =     "MVLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR"
horse =   "MVLSAADKTNVKAAWSKVGGHAGEYGAEALERMFLGFPTTKTYFPHF.DLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSSVSTVLTSKYR"  
donkey =  "MVLSAADKTNVKAAWSKVGGNAGEFGAEALERMFLGFPTTKTYFPHF.DLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSTVSTVLTSKYR" 
rabbit =  ".VLSPADKTNIKTAWEKIGSHGGEYGAEAVERMFLGFPTTKTYFPHF.DFTHGSZQIKAHGKKVSEALTKAVGHLDDLPGALSTLSDLHAHKLRVDPVNFKLLSHCLLVTLANHHPSEFTPAVHASLDKFLANVSTVLTSKYR"
carp =    "MSLSDKDKAAVKGLWAKISPKADDIGAEALGRMLTVYPQTKTYFAHWADLSPGSGPVKKHGKVIMGAVGDAVSKIDDLVGGLAALSELHAFKLRVDPANFKILAHNVIVVIGMLYPGDFPPEVHMSVDKFFQNLALALSEKYR"

A list of all of the hemoglobin sequences and a list of all of the nodes must be created so they can be used later on to find differences and to create the phenogenetic tree

In [ ]:
#create list of all hemoglobin sequences
animals = [carp, cow, donkey, horse, human, gorilla, rabbit]

#create list of all animal names and string to be used as nodes
nodes = ["carp", "cow", "donkey", "horse", "human", "gorilla", "rabbit"]

Then, a dictionary of all of the differences between each animal must be created. To do this, a function was created which took in the lists created above. It takes in the hemoglobin sequences and the list of animal names. Then, it uses the difference function above to determine the difference between each animal pair. Finally, it adds each animal pair to the difference dictionary and returns the dictionary

In [ ]:
def create_diff_dict(animals, nodes):
    '''Creates a matrix of differences between the given species.
    Stored in a dictionary of dictionary based on the indices of 
    the provided list.'''
    
    # Initialize difference matrix
    diff_dict = {}
    
    # Loop over all the animal pairs to find their differences
    for i, animal1 in enumerate(nodes):
        for j, animal2 in enumerate(nodes):
            
            # Don't check animals against themselves
            if(animal1 == animal2):
                continue
                
            diff = differences(animals[i], animals[j])
            
            # Update difference matrix with distance calculated
            if(not animal1 in diff_dict):
                diff_dict[animal1] = {}

            if(not animal2 in diff_dict):
                diff_dict[animal2] = {}

            diff_dict[animal1][animal2] = diff
            diff_dict[animal2][animal1] = diff
    
    # Return difference matrix
    return diff_dict

In [ ]:
create_diff_dict(animals, nodes)

Next, we need a function to find the closest two nodes that should be combined via the UPGMA algorithm

In [ ]:
def pair_group(diff_dict):
    """ given a matrix of differences, returns the indices of the closest two related organisms"""
    
    # Initialize what we're keeping track of
    min_val = np.inf
    index_1 = 0
    index_2 = 0
    
    # Loop over the distance matrix
    for i in diff_dict.keys():
        for j in diff_dict[i].keys():
            
            # Update our variables if we find a new minimum
            if diff_dict[i][j] < min_val:
                min_val = diff_dict[i][j]
                index_1 = i
                index_2 = j
    
    # Return the pair of indices that have the lowest score as a tuple
    return (index_1, index_2)

In [ ]:
pair_group(create_diff_dict(animals, nodes))

Since UPGMA looks at weighted averages, we need to figure out what the weight is of each node being combined. This function recursively loops over a tuple to count the total number of nodes stored in it.

In [ ]:
def calculate_weight(tup):
    '''Determine number of items in a nested tuple'''
    
    # Base case
    if(type(tup) != tuple):
        return 1
    
    num_values = 0
    
    # Loop over the tuple and unpack it based on its contents
    for i in tup:
        
        # Recursively determine weight if the item is another tuple
        if type(i) == tuple:
            num_values += calculate_weight(i)
            
        # Add 1 if the item is not a tuple
        else:
            num_values += 1
    
    # Return weight of tuple
    return num_values

In [ ]:
calculate_weight(((5, (3, 2)), ((1, 6), 5)))

UPGMA determines edge weights to the farthest node in a node group but we just want to update the edge weight between the new node and the node group as a whole. So, we need to figure out what the distance of the node group is in order to do this. Since all of the nodes will end up the same distance away from the root node, we only need to check one path to find the furthest node

In [ ]:
def get_distance_to_farthest_node(root):
    '''Gets the linear distance from a node to its furthest child'''
    
    # Base case
    if(root.right == None):
        return 0
    
    # Return the distance between this node and its child
    return root.right[1] + get_distance_to_farthest_node(root.right[0])

In [ ]:
root = Node()
root.right = (Node(), 5)
root.right[0].right = (Node(), 7)
root.right[0].right[0].right = (Node(), 10)

get_distance_to_farthest_node(root)

Now we're ready to apply our UPGMA algorithm to a matrix of differences. We first start by creating a dictionary of the base nodes so that our tree is a binary tree at the end. Then, we proceed by combining nodes into pair groups until there is only one node left which is the root of our phylogenetic tree.

In [ ]:
def generate_phylogenetic_tree(diff_dict):
    '''Creates a phylogentic tree from a list of sequences'''
    
    # Create the dictionary of nodes for the phylogentic tree creation
    node_dict = {}

    for species in diff_dict:
        node = Node()
        node.val = species
        node_dict[species] = node

    # Apply UPGMA until all nodes have been combined
    while(len(diff_dict.keys()) > 1):
        
        # Get the two most related nodes
        (animal_1, animal_2) = pair_group(diff_dict)
        
        # Add a new row for the new node
        new_group = (animal_1, animal_2)
        diff_dict[new_group] = {}
        
        # Get the distance between these two nodes
        distance = diff_dict[animal_1][animal_2]
        
        # Create a new Node representing the new group being formed. Include the edge weight between the nodes
        node = Node()
        node.val = new_group
        node.left = (node_dict[animal_1], distance / 2 - get_distance_to_farthest_node(node_dict[animal_1]))
        node.right = (node_dict[animal_2], distance / 2 - get_distance_to_farthest_node(node_dict[animal_2]))
        
        # Remove the old nodes and add the new ones to the node dictionary
        node_dict.pop(animal_1)
        node_dict.pop(animal_2)
        node_dict[new_group] = node
        
        # Delete node pairs from the distance matrix
        diff_dict[animal_1].pop(animal_2)
        diff_dict[animal_2].pop(animal_1)

        # Calculate the weights for the arithmetic mean
        weight1 = calculate_weight(animal_1)
        weight2 = calculate_weight(animal_2)

        # Populate new node's row
        for i in diff_dict[animal_1].keys():
            diff_dict[new_group][i] = (weight1 * diff_dict[animal_1][i] + weight2 * diff_dict[animal_2][i]) / (weight1 + weight2)

        # Delete old node's rows
        diff_dict.pop(animal_1)
        diff_dict.pop(animal_2)

        # Update remaining nodes' distances to the newly added node and delete the remaining
        # references to the old nodes
        for i in diff_dict.keys():
            if(i != new_group):
                diff_dict[i][new_group] = (weight1 * diff_dict[i][animal_1] + weight2 * diff_dict[i][animal_2]) / (weight1 + weight2)
                
                diff_dict[i].pop(animal_1)
                diff_dict[i].pop(animal_2)
    
    # The last created node is the completed tree
    return node

In [ ]:
# Create matrix of differences
diff_dict = create_diff_dict(animals, nodes)

# Create Node based tree
index_tree = generate_phylogenetic_tree(diff_dict)
index_tree.val

Looks like it all still works, so now let's put this data into a visualizable format. We will be using BioPython's Phylo module to display the tree which takes in a PhyloXML file, so we need to put the data into that format using the following two functions.

In [ ]:
def write_tree_xml(root, filename):
    '''Writes a weighted binary tree to a PhyloXML file'''
    
    # Open the file
    with open(filename + '.xml', 'w') as f:
        
        # Write the necessary header information
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write('<phyloxml xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.phyloxml.org http://www.phyloxml.org/1.10/phyloxml.xsd" xmlns="http://www.phyloxml.org">\n')
        f.write('<phylogeny rooted="true">\n')
        f.write('<clade>')
        
        # Recursively write each node to the file
        write_tree_recursive(root, f)
        
        # Close off the file
        f.write('</clade>\n')
        f.write('</phylogeny>\n')
        f.write('</phyloxml>\n')

In [ ]:
def write_tree_recursive(root, f):
    ''' Recursively adds clades to the file'''
    
    # Base case
    if(root.right == None):
        f.write('<name>' + root.val + '</name>\n')
        return
    
    # Add a new clade for the left child
    f.write('<clade branch_length="' + str(root.left[1]) + '">\n')
    write_tree_recursive(root.left[0], f)
    f.write('</clade>\n')
    
    # Add a new clade for the right child
    f.write('<clade branch_length="' + str(root.right[1]) + '">\n')
    write_tree_recursive(root.right[0], f)
    f.write('</clade>\n')

In [ ]:
# Generate PhyloXML file
write_tree_xml(index_tree, 'test_tree')

# Load and visualize phylogenetic tree
tree = Phylo.read('test_tree.xml', 'phyloxml')
tree.ladderize()
Phylo.draw(tree)

We got the same tree as before, so it seems like its working. We see that gorillas and humans are very closely related, which makes sense. While carp is very distantly related to any of the other species so the edge weights also seem to be working. Next, we need to apply this algorithm to our parsed codon biases.

# Part 3 - Creating a Tree of Rodents

Now that we have the UPGMA algorithm working, we can now apply it to our parsed rodent data. To do that, we first need a function that can create our differences matrix from a supplied dataframe.

In [ ]:
def create_diff_codon_dict(df):
    '''Creates a matrix of differences between the species in a 
    dataframe. Stored in a dictionary of dictionaries based on
    the names of the provided species.'''
    
    # Initialize difference matrix
    diff_dict = {}
    
    # Loop over all the animal pairs to find their differences
    for species1 in df['SpeciesName'].unique():
        for species2 in df['SpeciesName'].unique():
            
            # Ignore if the two species are the same
            if(species1 == species2):
                continue

            # Calculate the differences between the two species
            diff = species_diff(df, species1, species2)

            # Update the differences matrix
            if(not species1 in diff_dict):
                diff_dict[species1] = {}

            if(not species2 in diff_dict):
                diff_dict[species2] = {}

            diff_dict[species1][species2] = diff
            diff_dict[species2][species1] = diff
        
    # Return the differences matrix
    return diff_dict

In [ ]:
create_diff_codon_dict(rodents)

In [ ]:
# Create matrix of differences
diff_dict = create_diff_codon_dict(rodents)

# Create Node based tree
index_tree = generate_phylogenetic_tree(diff_dict)

# Generate PhyloXML file
write_tree_xml(index_tree, 'rodent_tree')

# Load and visualize phylogenetic tree
tree = Phylo.read('rodent_tree.xml', 'phyloxml')
tree.ladderize()
Phylo.draw(tree)

# Part 4 - Creating Tree of In Class Animals

We've got a tree built with species that are all pretty similar to one another. What happens if we try it with species that are more distantly related? First, a list with the scientific names of each animal in class was created so they could be found in the data frame. Carp was replaced with trout because carp was not in the data frame. It was assumed that the two fish should have a similar relation to the rest of the animals which are all mammals

In [ ]:
#create list of animal scientific names (carp replaced with trout)
animals = ['Homo sapiens', 'Gorilla gorilla', 'Oryctolagus cuniculus', 'Bos taurus', 'Equus asinus','Equus caballus', 'Oncorhynchus mykiss'] 

Next, a data frame had to be created for all of the in-class animals. First, the list of animals was looped through. If it was the first animal, a new data frame was created. If it was no, the new animal was added to the growing data frame using the pd.concat function which combines two data frames. Finally, the data frame was printed out to make sure it had been created properly. This could then be used in future steps

In [ ]:
#loop through list of animals
for animal in animals:
    
    #if first animal, create new data frame
    if animal == animals[0]:
        class_animals = df[df['SpeciesName'] == animal]
        
    #otherwise, add the new animal to the growing data frame
    else: 
        class_animals = pd.concat([class_animals, df[df['SpeciesName'] == animal]])

#print data frame to make sure it is all present
class_animals

The codon bias was standardized by amino acid. This was done in the same fashion as previously to create a data frame with codon bias for each amino acid

In [ ]:
#loop through each animal
for animal in class_animals['SpeciesName']:
    species_bool = class_animals['SpeciesName'] == animal
    species =  class_animals[species_bool]
    
    #loop through all amino acids
    for aa in aas:
        
        #initalize total percent of amino acid
        total = 0 
        
        #loop through codons and add up percentages
        for codon in aas[aa]:
            total += species[codon]
        
        #loop through codons, normalize by amino acid, and replace in data frame
        for codon in aas[aa]:
             class_animals.loc[species_bool, [codon]] = species[codon]/total

In [ ]:
#print the data frame to make sure each codon column has been correctly updated
class_animals

Once the data frame has been correctly created, it can be used to create a difference dictionary. This was done using the previously defined create_diff_codon_dict function. 

In [ ]:
#create dictionary of difference for class animals
create_diff_codon_dict(class_animals)

Finally, a tree can be created of all of the animals used in class. This was done the same was as previously done. 

In [ ]:
# Create matrix of differences
diff_dict = create_diff_codon_dict(class_animals)

# Create Node based tree
index_tree = generate_phylogenetic_tree(diff_dict)

# Generate PhyloXML file
write_tree_xml(index_tree, 'animal_tree')

# Load and visualize phylogenetic tree
tree = Phylo.read('animal_tree.xml', 'phyloxml')
tree.ladderize()
Phylo.draw(tree)

# Results and Discussion

# Next Steps

Future steps in this project could explore this method with a larger data set to confirm if the results hold true. The mitocondiral data could be used to see if that would be able to create a more accurate phenogenetic tree. Mitocondrial DNA could also be compared to bacteria to track the relationship between mitocondria and modern bacteria. Finally, the discripancies in the tree created could be explored to determine the cause of the discripancies